# Converting the Raw dataset to the comma seperated file for model building

In [1]:
# import csv

# txt_file = 'Smarthome.rawdata.txt'
# csv_file = 'Smarthome.rawdata.csv'

# with open(txt_file, 'r') as file:
#     lines = file.readlines()

# data = []
# for line in lines:
#     # Split values using spaces as the delimiter
#     values = line.strip().split()
#     data.append(values)

# with open(csv_file, 'w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerows(data)

# print("Conversion complete. Output saved as", csv_file)


Conversion complete. Output saved as Smarthome_rawdata.csv


In [16]:
import pandas as pd

In [17]:
path = "Smarthome.rawdata.csv"
df = pd.read_csv(path)
df.head()

,Date,Time,Sensor,Room Name,Room Description,Device Status,Sensor-Message
0,6/15/2011,17:04.3,LS013,Ignore,Ignore,1,Control4-LightSensor
1,6/15/2011,24:14.7,LS001,Ignore,Ignore,3,Control4-LightSensor
2,6/15/2011,34:14.7,LS001,Ignore,Ignore,4,Control4-LightSensor
3,6/15/2011,37:04.2,LS013,Ignore,Ignore,2,Control4-LightSensor
4,6/15/2011,50:38.2,LS012,Ignore,Ignore,42,Control4-LightSensor


In [18]:
df.columns

Index(['Date', 'Time', 'Sensor', 'Room Name', 'Room Description',
       'Device Status', 'Sensor-Message'],
      dtype='object')

In [19]:
df.count

<bound method DataFrame.count of               Date     Time Sensor Room Name Room Description Device Status  \
0        6/15/2011  17:04.3  LS013    Ignore           Ignore             1   
1        6/15/2011  24:14.7  LS001    Ignore           Ignore             3   
2        6/15/2011  34:14.7  LS001    Ignore           Ignore             4   
3        6/15/2011  37:04.2  LS013    Ignore           Ignore             2   
4        6/15/2011  50:38.2  LS012    Ignore           Ignore            42   
...            ...      ...    ...       ...              ...           ...   
1048570  12/6/2011  24:00.6   M003   Kitchen          Kitchen            ON   
1048571  12/6/2011  24:06.6   M003   Kitchen          Kitchen           OFF   
1048572  12/6/2011  24:07.3   M003   Kitchen          Kitchen            ON   
1048573  12/6/2011  24:10.9   M003   Kitchen          Kitchen           OFF   
1048574  12/6/2011  24:12.2   M003   Kitchen          Kitchen            ON   

               Sen

In [20]:
# Check the data types of variables
df.dtypes

Date                object
Time                object
Sensor              object
Room Name           object
Room Description    object
Device Status       object
Sensor-Message      object
dtype: object

# Dropping the duplicate values

In [21]:
# Drop duplicates
df = df.drop_duplicates()


In [22]:

# Check for missing values
print(df.isnull().sum())

Date                0
Time                0
Sensor              0
Room Name           0
Room Description    0
Device Status       0
Sensor-Message      0
dtype: int64


In [23]:
# Get the summary statistics of the dataset
df.describe()

,Date,Time,Sensor,Room Name,Room Description,Device Status,Sensor-Message
count,1045674,1045674,1045674,1045674,1045674,1045674,1045674
unique,175,36000,56,6,11,105,7
top,10/2/2011,27:19.7,MA004,Kitchen,Kitchen,OFF,Control4-MotionArea
freq,12919,56,185367,370203,370203,374067,510053


In [24]:
pip install pymongo


Note: you may need to restart the kernel to use updated packages.


# Display the List of rooms

In [25]:
import pymongo

# Connect to MongoDB
client = pymongo.MongoClient('mongodb://localhost:27017')
database = client['smart_home']
collection = database['smart_home_raw_data']

# Fetch distinct room names from the collection
room_names = collection.distinct("Room Name")

# Display the room names
for room_name in room_names:
    print(room_name)

# Close the MongoDB connection
client.close()


Bathroom
Bedroom
Kitchen
LivingRoom
OutsideDoor


In [26]:
# import pymongo

# # Connect to MongoDB
# client = pymongo.MongoClient('mongodb://localhost:27017')
# database = client['smart_home']
# collection = database['smart_home_raw_data']

# # Fetch documents from the collection
# query = {"Room Name": "Bathroom"}
# documents = collection.find(query)

# # Display document values
# for document in documents:
#     print(document)

# # Close the MongoDB connection
# client.close()


# Count of all the rooms and devices

In [27]:
import pymongo

# Connect to MongoDB
client = pymongo.MongoClient('mongodb://localhost:27017')
database = client['smart_home']
collection = database['smart_home_raw_data']

# Perform aggregation to get room count
pipeline = [
    {
        '$group': {
            '_id': '$Room Name',
            'count': {'$sum': 1}
        }
    }
]
result = collection.aggregate(pipeline)

# Display the room count
for entry in result:
    room_name = entry['_id']
    count = entry['count']
    print(f"Room Name: {room_name}, Count: {count}")

# Close the MongoDB connection
client.close()


Room Name: LivingRoom, Count: 212971
Room Name: Bathroom, Count: 43470
Room Name: Bedroom, Count: 118226
Room Name: Kitchen, Count: 372225
Room Name: OutsideDoor, Count: 15882


# Device Configuration

In [28]:
import pymongo

# Connect to MongoDB
client = pymongo.MongoClient('mongodb://localhost:27017')
database = client['smart_home']
collection = database['smart_home_raw_data']

# Perform aggregation to get sensors linked to each room
pipeline = [
    {
        '$group': {
            '_id': '$Room Name',
            'sensors': {'$addToSet': '$Sensor'}
        }
    }
]
result = collection.aggregate(pipeline)

# Display the sensors linked to each room
for entry in result:
    room_name = entry['_id']
    sensors = entry['sensors']
    print(f"Room Name: {room_name}, Sensors: {sensors}")

# Close the MongoDB connection
client.close()


Room Name: Bedroom, Sensors: ['MA011', 'L006', 'MA012', 'MA010']
Room Name: OutsideDoor, Sensors: ['M001', 'D002', 'L001']
Room Name: Bathroom, Sensors: ['D003', 'L007', 'L008', 'MA013']
Room Name: LivingRoom, Sensors: ['L005', 'MA009', 'MA006', 'MA008', 'M005', 'MA007']
Room Name: Kitchen, Sensors: ['M002', 'L002', 'L004', 'L003', 'MA004', 'M003']


In [29]:
import pymongo

# Connect to MongoDB
client = pymongo.MongoClient('mongodb://localhost:27017')
database = client['smart_home']
collection = database['smart_home_raw_data']

# Remove documents where Room Name is "Ignore"
collection.delete_many({"Room Name": "Ignore"})

# Get distinct room names from the collection
available_rooms = collection.distinct("Room Name")

# Display the available rooms
for room in available_rooms:
    print(room)

# Close the MongoDB connection
client.close()


Bathroom
Bedroom
Kitchen
LivingRoom
OutsideDoor


In [30]:
# import pymongo

# # Connect to MongoDB
# client = pymongo.MongoClient('mongodb://localhost:27017')
# database = client['smart_home']
# collection = database['smart_home_raw_data']

# # Perform aggregation
# pipeline = [
#     {"$match": {"Room Name": {"$ne": "Ignore"}}},
#     {"$group": {"_id": "$Room Name", "Sensors": {"$push": "$Sensor"}}},
#     {"$limit": 10}  # Limit the output to the first 10 rooms
# ]
# result = list(collection.aggregate(pipeline))

# # Display the sensors linked to each room
# for item in result:
#     room = item["_id"]
#     sensors = item["Sensors"]
#     print(f"Room: {room}")
#     print(f"Sensors: {sensors}")
#     print()

# # Close the MongoDB connection
# client.close()


In [37]:
# import pymongo
# import pandas as pd

# # Connect to MongoDB
# client = pymongo.MongoClient('mongodb://localhost:27017')
# database = client['smart_home']
# collection = database['smart_home_raw_data']

# # Perform aggregation
# pipeline = [
#     {"$match": {"Room Name": {"$ne": "Ignore"}}},
#     {"$group": {"_id": {"Room": "$Room Name", "Date": "$Date"}, "Sensors": {"$push": "$Sensor"}}},
#     {"$limit": 10}  # Limit the output to the first 10 rooms
# ]
# result = list(collection.aggregate(pipeline))

# # Fetch all required data from the database
# room_names = [item["_id"]["Room"] for item in result]
# dates = [item["_id"]["Date"] for item in result]
# all_sensors = [sensor for item in result for sensor in item["Sensors"]]

# device_status_map = {}
# for room in room_names:
#     for date in dates:
#         query = {"Room Name": room, "Date": date}
#         cursor = collection.find(query, {"Sensor": 1, "Device Status": 1})
#         for doc in cursor:
#             sensor_name = doc["Sensor"]
#             device_status = doc.get("Device Status", "Unknown")
#             device_status_map[(room, date, sensor_name)] = device_status

# # Prepare report data
# report_data = []
# for item in result:
#     room = item["_id"]["Room"]
#     date = item["_id"]["Date"]
#     sensors = item["Sensors"]

#     for sensor in sensors:
#         device_status = device_status_map.get((room, date, sensor), "Unknown")
#         report_data.append({"Date": date, "Time": "", "Sensor": sensor, "Room Name": room, "Room Description": "", "Device Status": device_status, "Sensor-Message": ""})

# # Create a DataFrame from the report data
# df = pd.DataFrame(report_data)

# # Generate Excel report
# report_file = "sensor_report.xlsx"
# df.to_excel(report_file, index=False)

# # Close the MongoDB connection
# client.close()

# print("Report generated successfully.")


Report generated successfully.


In [41]:
# Connect to MongoDB
client = pymongo.MongoClient('mongodb://localhost:27017')
database = client['smart_home']
collection = database['smart_home_raw_data']

# Perform aggregation
pipeline = [
    {"$group": {"_id": "$Device Status", "count": {"$sum": 1}}},
    {"$match": {"_id": {"$in": ["CLOSE", "OFF", "ON", "OPEN"]}}}
]
result = list(collection.aggregate(pipeline))

# Display the count of each device status
for item in result:
    status = item["_id"]
    count = item["count"]
    print(f"Device Status: {status}, Count: {count}")

# Close the MongoDB connection
client.close()



Device Status: ON, Count: 375460
Device Status: OFF, Count: 375481
Device Status: OPEN, Count: 1624
Device Status: CLOSE, Count: 1620


In [5]:
import pymongo
import pandas as pd

# Connect to MongoDB
client = pymongo.MongoClient('mongodb://localhost:27017')
database = client['smart_home']
collection = database['smart_home_raw_data']

# Perform aggregation
pipeline = [
    {"$match": {"Room Name": {"$ne": "Ignore"}}},
    {"$group": {"_id": {"Room": "$Room Name", "Date": "$Date"}, "Sensors": {"$push": "$Sensor"}}},
    {"$limit": 10}  # Limit the output to the first 10 rooms
]
result = list(collection.aggregate(pipeline))

# Fetch all required data from the database
room_names = [item["_id"]["Room"] for item in result]
dates = [item["_id"]["Date"] for item in result]
all_sensors = [sensor for item in result for sensor in item["Sensors"]]

device_status_map = {}
for room in room_names:
    for date in dates:
        query = {"Room Name": room, "Date": date}
        cursor = collection.find(query, {"Sensor": 1, "Device Status": 1, "Time": 1, "Room Description": 1, "Sensor-Message": 1})
        for doc in cursor:
            sensor_name = doc["Sensor"]
            device_status = doc.get("Device Status", "Unknown")
            if device_status in ["CLOSE", "OFF", "ON", "OPEN"]:
                time = doc.get("Time", "")
                room_description = doc.get("Room Description", "")
                sensor_message = doc.get("Sensor-Message", "")
                device_status_map[(room, date, sensor_name)] = {
                    "Device Status": device_status,
                    "Time": time,
                    "Room Description": room_description,
                    "Sensor-Message": sensor_message
                }

# Prepare report data
report_data = []
for item in result:
    room = item["_id"]["Room"]
    date = item["_id"]["Date"]
    sensors = item["Sensors"]

    for sensor in sensors:
        sensor_data = device_status_map.get((room, date, sensor), {})
        device_status = sensor_data.get("Device Status", "Unknown")
        time = sensor_data.get("Time", "")
        room_description = sensor_data.get("Room Description", "")
        sensor_message = sensor_data.get("Sensor-Message", "")
        report_data.append({"Date": date, "Time": time, "Sensor": sensor, "Room Name": room, "Room Description": room_description, "Device Status": device_status, "Sensor-Message": sensor_message})

# Create a DataFrame from the report data
df = pd.DataFrame(report_data)

# Generate Excel report
report_file = "sensor_report.xlsx"
df.to_excel(report_file, index=False)

# Close the MongoDB connection
client.close()

print("Report generated successfully.")


Report generated successfully.
